In [1]:
import pandas as pd

df = pd.read_csv('./classify_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354667 entries, 0 to 354666
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   score       354667 non-null  float64
 1   hashtag     354667 non-null  object 
 2   lang        354667 non-null  object 
 3   tweet_lang  354667 non-null  object 
 4   time_zone   354667 non-null  object 
 5   created_at  354667 non-null  object 
 6   season      354667 non-null  int64  
 7   month       354667 non-null  int64  
 8   day_time    354667 non-null  object 
 9   hours       354667 non-null  int64  
 10  track_id    354667 non-null  object 
 11  valence     354667 non-null  float64
dtypes: float64(2), int64(3), object(7)
memory usage: 32.5+ MB


In [2]:
track_to_label = pd.read_csv('./track_to_labels.csv')
track_to_label.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26106 entries, 0 to 26105
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   track_id  26106 non-null  object
 1   labels    26106 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 408.0+ KB


In [3]:
df = df.merge(track_to_label, on='track_id', how='left')
df.head()

,score,hashtag,lang,tweet_lang,time_zone,created_at,season,month,day_time,hours,track_id,valence,labels
0,0.8,kiss92,en,en,Beijing,2014-01-01 05:59:38,0,1,Wee hours(00:00 - 6:00),5,b129f3e0a0bdc1e61c57ce994f9d2932,0.320,0
1,0.8,kiss92,en,en,Beijing,2014-01-01 06:06:15,0,1,Morning(6:00 - 12:00),6,64b30f6ac1acecb7cef9dc890e1c3ea7,0.380,2
2,0.8,kiss92,en,en,Beijing,2014-01-01 06:11:07,0,1,Morning(6:00 - 12:00),6,cc1a46ee0446538ecf6b65db01c30cd8,0.270,2
3,0.8,kiss92,en,en,Beijing,2014-01-01 06:19:43,0,1,Morning(6:00 - 12:00),6,dbbd8525c1bc48cc791bfe618b74bac6,0.576,4
4,0.8,kiss92,en,en,Beijing,2014-01-01 06:23:47,0,1,Morning(6:00 - 12:00),6,dbdbb866939c1ab1dfd0d5cfac740eaa,0.768,4


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354667 entries, 0 to 354666
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   score       354667 non-null  float64
 1   hashtag     354667 non-null  object 
 2   lang        354667 non-null  object 
 3   tweet_lang  354667 non-null  object 
 4   time_zone   354667 non-null  object 
 5   created_at  354667 non-null  object 
 6   season      354667 non-null  int64  
 7   month       354667 non-null  int64  
 8   day_time    354667 non-null  object 
 9   hours       354667 non-null  int64  
 10  track_id    354667 non-null  object 
 11  valence     354667 non-null  float64
 12  labels      354667 non-null  int64  
dtypes: float64(2), int64(4), object(7)
memory usage: 35.2+ MB


In [5]:
df.labels.value_counts()

labels
4    96195
2    86444
1    66020
3    59237
0    46771
Name: count, dtype: int64

In [6]:
# 去除离群点
df = df[df['labels'] != -1]
df.labels.value_counts()

labels
4    96195
2    86444
1    66020
3    59237
0    46771
Name: count, dtype: int64

In [7]:
df.head()

,score,hashtag,lang,tweet_lang,time_zone,created_at,season,month,day_time,hours,track_id,valence,labels
0,0.8,kiss92,en,en,Beijing,2014-01-01 05:59:38,0,1,Wee hours(00:00 - 6:00),5,b129f3e0a0bdc1e61c57ce994f9d2932,0.320,0
1,0.8,kiss92,en,en,Beijing,2014-01-01 06:06:15,0,1,Morning(6:00 - 12:00),6,64b30f6ac1acecb7cef9dc890e1c3ea7,0.380,2
2,0.8,kiss92,en,en,Beijing,2014-01-01 06:11:07,0,1,Morning(6:00 - 12:00),6,cc1a46ee0446538ecf6b65db01c30cd8,0.270,2
3,0.8,kiss92,en,en,Beijing,2014-01-01 06:19:43,0,1,Morning(6:00 - 12:00),6,dbbd8525c1bc48cc791bfe618b74bac6,0.576,4
4,0.8,kiss92,en,en,Beijing,2014-01-01 06:23:47,0,1,Morning(6:00 - 12:00),6,dbdbb866939c1ab1dfd0d5cfac740eaa,0.768,4


In [8]:
# 创建交叉组合特征
df['month_day_time'] = df['month'].astype(str) + '_' + df['day_time']
df['day_time_hours'] = df['day_time'] + '_' + df['hours'].astype(str)

# 查看结果
print(df.head())

   score hashtag lang tweet_lang time_zone           created_at  season  \
0    0.8  kiss92   en         en   Beijing  2014-01-01 05:59:38       0   
1    0.8  kiss92   en         en   Beijing  2014-01-01 06:06:15       0   
2    0.8  kiss92   en         en   Beijing  2014-01-01 06:11:07       0   
3    0.8  kiss92   en         en   Beijing  2014-01-01 06:19:43       0   
4    0.8  kiss92   en         en   Beijing  2014-01-01 06:23:47       0   

   month                 day_time  hours                          track_id  \
0      1  Wee hours(00:00 - 6:00)      5  b129f3e0a0bdc1e61c57ce994f9d2932   
1      1    Morning(6:00 - 12:00)      6  64b30f6ac1acecb7cef9dc890e1c3ea7   
2      1    Morning(6:00 - 12:00)      6  cc1a46ee0446538ecf6b65db01c30cd8   
3      1    Morning(6:00 - 12:00)      6  dbbd8525c1bc48cc791bfe618b74bac6   
4      1    Morning(6:00 - 12:00)      6  dbdbb866939c1ab1dfd0d5cfac740eaa   

   valence  labels             month_day_time             day_time_hours  
0    

In [9]:
# import torch
# import torch.nn as nn

# num_categories = 24  # 示例：10个类别
# embedding_dim = 5  # 嵌入向量的维度

# # 创建嵌入层
# embedding_layer = nn.Embedding(num_categories, embedding_dim)

# # 示例：对一批类别数据进行嵌入
# category_data = torch.tensor(df['hours'], dtype=torch.long)  # 假设的类别数据
# embeddings = embedding_layer(category_data)  # 获取嵌入向量

# print(embeddings)  # 输出嵌入向量


In [10]:
# 对情绪分数score进行分箱操作
labels = ['很悲伤', '悲伤', '平静', '开心', '很开心']
bins = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]
df['mood'] = pd.cut(df['score'], labels=labels, bins = bins, include_lowest=True)
df['mood_label'] = df['mood'].cat.codes
df.drop(['mood'], axis = 1, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354667 entries, 0 to 354666
Data columns (total 16 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   score           354667 non-null  float64
 1   hashtag         354667 non-null  object 
 2   lang            354667 non-null  object 
 3   tweet_lang      354667 non-null  object 
 4   time_zone       354667 non-null  object 
 5   created_at      354667 non-null  object 
 6   season          354667 non-null  int64  
 7   month           354667 non-null  int64  
 8   day_time        354667 non-null  object 
 9   hours           354667 non-null  int64  
 10  track_id        354667 non-null  object 
 11  valence         354667 non-null  float64
 12  labels          354667 non-null  int64  
 13  month_day_time  354667 non-null  object 
 14  day_time_hours  354667 non-null  object 
 15  mood_label      354667 non-null  int8   
dtypes: float64(2), int64(4), int8(1), object(9)
memory usage

In [11]:
# One-hot编码
encode_columns = ['day_time', 'month_day_time', 'day_time_hours'] 
df = pd.get_dummies(df, columns=encode_columns, dtype='float32')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354667 entries, 0 to 354666
Data columns (total 89 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   score                                       354667 non-null  float64
 1   hashtag                                     354667 non-null  object 
 2   lang                                        354667 non-null  object 
 3   tweet_lang                                  354667 non-null  object 
 4   time_zone                                   354667 non-null  object 
 5   created_at                                  354667 non-null  object 
 6   season                                      354667 non-null  int64  
 7   month                                       354667 non-null  int64  
 8   hours                                       354667 non-null  int64  
 9   track_id                                    354667 non-null  object 
 

In [12]:
df.drop(['created_at'], axis = 1, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354667 entries, 0 to 354666
Data columns (total 88 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   score                                       354667 non-null  float64
 1   hashtag                                     354667 non-null  object 
 2   lang                                        354667 non-null  object 
 3   tweet_lang                                  354667 non-null  object 
 4   time_zone                                   354667 non-null  object 
 5   season                                      354667 non-null  int64  
 6   month                                       354667 non-null  int64  
 7   hours                                       354667 non-null  int64  
 8   track_id                                    354667 non-null  object 
 9   valence                                     354667 non-null  float64
 

In [13]:
# 对几个类别特别多的列采用二进制编码
from category_encoders import BinaryEncoder
# 二进制编码
columns = ['hashtag', 'tweet_lang', 'time_zone', 'lang']
for column in columns:
    encoder = BinaryEncoder(cols=[column])
    df = encoder.fit_transform(df)

print(df.info())
# 查看结果

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354667 entries, 0 to 354666
Columns: 114 entries, score to day_time_hours_Wee hours(00:00 - 6:00)_5
dtypes: float32(76), float64(2), int64(34), int8(1), object(1)
memory usage: 203.3+ MB
None


In [14]:
# 清理列名，删除或替换特殊字符
import re
df.columns = [re.sub(r'\([^)]*\)', '', col) for col in df.columns]
df.columns

Index(['score', 'hashtag_0', 'hashtag_1', 'hashtag_2', 'hashtag_3',
       'hashtag_4', 'hashtag_5', 'hashtag_6', 'hashtag_7', 'hashtag_8',
       ...
       'day_time_hours_Night_20', 'day_time_hours_Night_21',
       'day_time_hours_Night_22', 'day_time_hours_Night_23',
       'day_time_hours_Wee hours_0', 'day_time_hours_Wee hours_1',
       'day_time_hours_Wee hours_2', 'day_time_hours_Wee hours_3',
       'day_time_hours_Wee hours_4', 'day_time_hours_Wee hours_5'],
      dtype='object', length=114)

### 分类

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
import numpy as np

    
X = df.drop(['track_id','labels', 'valence', 'month'], axis = 1)
y = df['labels']

for column in X.columns:
    X[column] = X[column].astype(np.float32)
    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42, shuffle=True)


print(X_train.info())

<class 'pandas.core.frame.DataFrame'>
Index: 283733 entries, 271215 to 121958
Columns: 110 entries, score to day_time_hours_Wee hours_5
dtypes: float32(110)
memory usage: 121.2 MB
None


In [16]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)
print(X_train)
print(X_train.shape)

[[0.76666665 0.         0.         ... 0.         0.         0.        ]
 [0.8        0.         0.         ... 0.         1.         0.        ]
 [0.675      0.         0.         ... 0.         0.         0.        ]
 ...
 [0.76666665 0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.675      0.         0.         ... 0.         0.         0.        ]]
(283733, 110)


In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # 每个样本是一个序列，我们可以考虑每个样本的特征为序列中的单个步骤
        return self.features[idx].unsqueeze(0), self.labels[idx]  # 添加序列维度

class TransformerModel(nn.Module):
    def __init__(self, feature_size, num_classes=5, num_heads=2, num_layers=2, dropout=0.1):
        super(TransformerModel, self).__init__()
        self.embed = nn.Linear(feature_size, feature_size)
        encoder_layer = nn.TransformerEncoderLayer(d_model=feature_size, nhead=num_heads, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.output = nn.Linear(feature_size, num_classes)

    def forward(self, src):
        src = self.embed(src)  # 嵌入层调整特征大小
        src = src.permute(1, 0, 2)  # 调整维度以符合 Transformer 输入要求：(seq_len, batch, feature)
        output = self.transformer_encoder(src)
        output = output.mean(dim=0)  # 在序列长度上进行平均
        output = self.output(output)
        return output


In [21]:
# 转换为张量
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 创建 DataLoader
train_data = CustomDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

# 创建模型
model = TransformerModel(feature_size=110, num_classes=5)
model.to(device)
criterion = nn.CrossEntropyLoss()
criterion.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.002)

# 训练模型
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    print(f'---------------------------第{epoch+1}轮训练开始---------------------------')
    total, correct = 0, 0
    for data, target in train_loader:
        optimizer.zero_grad()
        data, target = data.to(device), target.to(device)
        output = model(data)
        
        # 计算准确率
        _, predicted = torch.max(output, 1)
        correct += (predicted == target).sum().item()  # 确保转换为 Python int
        total += target.size(0)  # total 累加 target 的数量
        
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'Train_set Loss: {total_loss / len(train_loader)} Train_set Accuracy:{100 * correct / total}')
    
    model.eval()
    with torch.no_grad():
        test_data = X_test_tensor.unsqueeze(1).permute(1, 0, 2).to(device) 
        outputs = model(test_data)  # 调整测试数据的形状
        predicted = outputs.argmax(dim=1)
        accuracy = (predicted == y_test_tensor.to(device)).float().mean()
        print(f'Test Accuracy: {accuracy.item()}')

---------------------------第1轮训练开始---------------------------
Train_set Loss: 1.5190148801939105 Train_set Accuracy:35.19928947284947
Test Accuracy: 0.27140721678733826
---------------------------第2轮训练开始---------------------------
Train_set Loss: 1.4918683039721257 Train_set Accuracy:37.15887824116335
Test Accuracy: 0.27140721678733826
---------------------------第3轮训练开始---------------------------
Train_set Loss: 1.4938724553207272 Train_set Accuracy:36.79973778164683
Test Accuracy: 0.27140721678733826
---------------------------第4轮训练开始---------------------------
Train_set Loss: 1.4747213152923833 Train_set Accuracy:37.17967243852495
Test Accuracy: 0.24388867616653442
---------------------------第5轮训练开始---------------------------
Train_set Loss: 1.4602482626211473 Train_set Accuracy:37.45845566077968
Test Accuracy: 0.27140721678733826
---------------------------第6轮训练开始---------------------------
Train_set Loss: 1.4533541225534665 Train_set Accuracy:37.80702279960385
Test Accuracy: 0.2714

Train_set Loss: 1.4121092481409883 Train_set Accuracy:39.709163192156005
Test Accuracy: 0.1856655478477478
---------------------------第50轮训练开始---------------------------
Train_set Loss: 1.4115693970760148 Train_set Accuracy:39.746874702625355
Test Accuracy: 0.27140721678733826
---------------------------第51轮训练开始---------------------------


KeyboardInterrupt: 

In [23]:
torch.save(model, './models/transformer.pth')

In [24]:
models = [RandomForestClassifier(n_estimators=10, random_state=42),
          LGBMClassifier(force_row_wise=True),
          DecisionTreeClassifier(random_state=42), 
          LogisticRegression()
#           KNeighborsClassifier(n_neighbors=3),
#           LinearSVC(random_state=42),
         ]
for model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"{model.__class__.__name__} Accuracy:", accuracy_score(y_test, y_pred))

RandomForestClassifier Accuracy: 0.3949445963853723
[LightGBM] [Info] Total Bins 367
[LightGBM] [Info] Number of data points in the train set: 283733, number of used features: 110
[LightGBM] [Info] Start training from score -2.026737
[LightGBM] [Info] Start training from score -1.680576
[LightGBM] [Info] Start training from score -1.411842
[LightGBM] [Info] Start training from score -1.789200
[LightGBM] [Info] Start training from score -1.304969
LGBMClassifier Accuracy: 0.40805537541940395
DecisionTreeClassifier Accuracy: 0.39524064623452787
LogisticRegression Accuracy: 0.36380297177658105


/home/s1751118/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### 回归

In [25]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score

X = df.drop(['track_id','labels', 'valence'], axis = 1)
y = df['valence']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

models = [LinearRegression(), Ridge(alpha=1.0), Lasso(alpha=0.1)]
for model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    mean_squared = mean_squared_error(y_test, y_pred)
    sqrt_mean_squared = np.sqrt(mean_squared)
    mean_absolute = mean_absolute_error(y_test, y_pred)
    r2score = r2_score(y_test, y_pred)
    print(model.__class__.__name__)
    print(f'MSE : {mean_squared}')
    print(f'RMSE : {sqrt_mean_squared}')
    print(f'MAE : {mean_absolute}')
    print(f'R² Score : {r2score}')

LinearRegression
MSE : 0.054108477379216306
RMSE : 0.2326122898284102
MAE : 0.19492192752014362
R² Score : 0.11812262087272485
Ridge
MSE : 0.054104237282757345
RMSE : 0.2326031755646456
MAE : 0.1949072513974091
R² Score : 0.11819172732948702
Lasso
MSE : 0.06135677692551016
RMSE : 0.24770300144630902
MAE : 0.2104832632031657
R² Score : -1.249799628855186e-05
